In [1]:
# Import Tools
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

# Import Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Other
import warnings
warnings.filterwarnings("ignore")
import zipfile
import os
from ds100_utils import run_linear_regression_test

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

# Get Data
with zipfile.ZipFile('cook_county_data.zip') as item:
    item.extractall()


In [13]:

# Here I'll just be making sure that the data is in good shape.

training_data = pd.read_csv("cook_county_train.csv", index_col='Unnamed: 0')

assert training_data.shape == (204758, 62)
# We will use a simple linear model with Sale Price as our regressand. Lets make sure it is here.
assert 'Sale Price' in training_data.columns.values

# For those interested, run the command below. These are going to be the features we have access to select from that are included in the data.

# training_data.columns

In [ ]:
# Next, I'll start my analysis